In [ ]:
import sys
pt_models = '../input/pretrainedmodels/pretrainedmodels-0.7.4/'
sys.path.insert(0, pt_models)

In [ ]:
import pretrainedmodels

In [ ]:
import glob
import torch
import albumentations
import joblib
import pandas as pd
import numpy as np
import torch.nn as nn

from tqdm import tqdm
from PIL import Image
from torch.nn import functional as F

In [ ]:
TEST_BATCH_SIZE = 32
MODEL_MEAN = (0.485, 0.456, 0.406)
MODEL_STD = (0.229, 0.224, 0.225)
IMG_HEIGHT = 28
IMG_WIDTH = 28
DEVICE="cuda"

In [ ]:
class resnet34(nn.Module):
    def __init__(self, pretrained):
        super(resnet34, self).__init__()
        if pretrained:
            self.model = pretrainedmodels.__dict__["resnet34"](pretrained="imagenet")
        else:
            self.model = pretrainedmodels.__dict__["resnet34"](pretrained=None)

        self.out = nn.Linear(512, 10)

    def forward(self, x):  # Takes a batch
        bs, channels, height, width = x.shape
        x = self.model.features(x)  # function of pretrainedmodels

        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        out = self.out(x)

        return out

In [ ]:
class KannadaMNISTTest:
    def __init__(self, df, img_width, img_height, mean, std):

        self.image_ids = df.id.values
        self.img_arr = df.iloc[:, 1:].values

        # Augmentations

        self.augment = albumentations.Compose(
            [
                albumentations.Resize(img_height, img_width, always_apply=True),
                albumentations.Normalize(mean, std, always_apply=True),
            ]
        )

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, item):
        image = self.img_arr[item, :]
        image = image.reshape(28, 28).astype(float)
        image = Image.fromarray(image).convert("RGB")  # WHY?
        id = self.image_ids[item]
        # Because all the models that we would try:
        # maybe from torchvision or pretrainedmodels they all work on rgb.
        # So we don't want to spend time on making them work for single channel only

        image = self.augment(image=np.array(image))["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)

        # Take a look at torchvision models to know why such dtype
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "id": torch.tensor(id, dtype=torch.long)
        }

In [ ]:
model = resnet34(pretrained=False)
model = model.to(DEVICE)

In [ ]:
model.load_state_dict(torch.load('../input/resnet34kannadamnist/model-resnet34-fold-4-epoch-7.bin'))

In [ ]:
model.eval()

In [ ]:
test_df = pd.read_csv('../input/Kannada-MNIST/test.csv')

In [ ]:
test_dataset = KannadaMNISTTest(df=test_df, img_width=28, img_height=28,
                                mean=MODEL_MEAN,
                                std=MODEL_STD)

In [ ]:
dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64,
    shuffle=False,
    num_workers=6
)

In [ ]:
predictions = []
for bt, d in tqdm(enumerate(dataloader), total = int(len(test_dataset) / dataloader.batch_size)):
    image = d['image']
    id = d['id']
    image = image.to(DEVICE, dtype=torch.float)
    out = model(image)
    digit = np.argmax(out.cpu().detach().numpy(), axis=1)
#     print(id, digit)
    for ii, imid in enumerate(id):
        predictions.append([int(imid.cpu().detach().numpy()), digit[ii]])

In [ ]:
df_sub = pd.DataFrame(predictions, columns=['id', 'label'])

In [ ]:
df_sub.head()

In [ ]:
df_sub.to_csv("submission.csv", index=False, columns=df_sub.columns)